In [1]:
#!uv pip install -qU langchain langchain-community langchain-experimental langchain-openai langchain-qdrant langchain-cohere rank_bm25 qdrant-client langsmith

In [2]:
from pathlib import Path
import requests
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["COHERE_API_KEY"] = os.getenv('COHERE_API_KEY')

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "retrieval-method-comparison"
os.environ["LANGSMITH_API_KEY"] = os.getenv('LANGSMITH_API_KEY')

QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
QDRANT_API_URL = os.getenv("QDRANT_API_URL")

In [3]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(model="gpt-4.1-mini")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [4]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### Data Preparation

In [5]:
"""### Data Preparation"""

# Set up a consistent data directory in the user's home directory
DATA_DIR = Path.home() / "data"
DATA_DIR.mkdir(exist_ok=True)

# URLs and filenames
urls = [
    ("https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv", "john_wick_1.csv"),
    ("https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv", "john_wick_2.csv"),
    ("https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv", "john_wick_3.csv"),
    ("https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv", "john_wick_4.csv"),
]

# Download files if not already present
for url, fname in urls:
    file_path = DATA_DIR / fname
    if not file_path.exists():
        print(f"Downloading {fname}...")
        r = requests.get(url)
        r.raise_for_status()
        file_path.write_bytes(r.content)
    else:
        print(f"{fname} already exists.")

john_wick_1.csv already exists.
john_wick_2.csv already exists.
john_wick_3.csv already exists.
john_wick_4.csv already exists.


In [6]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

parent_docs = documents

In [7]:
# display length of parent_docs and docs
print(f"Length of parent_docs: {len(parent_docs)}")
print(f"Length of documents: {len(documents)}")

Length of parent_docs: 100
Length of documents: 100


## Setup Vector Stores

In [8]:
import os
from langchain_qdrant import QdrantVectorStore  # Updated import
from langchain_openai import OpenAIEmbeddings
from qdrant_client import QdrantClient, models
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

In [9]:
# 1. Main vectorstore using qdrant cloud
baseline_vectorstore = QdrantVectorStore.from_documents(
    documents,
    embeddings,
    url=QDRANT_API_URL,
    api_key=QDRANT_API_KEY,
    prefer_grpc=True,
    collection_name="johnwick_baseline"
)

In [10]:
# 2. Parent document setup with qdrant cloud client

# Initialize cloud client
cloud_client = QdrantClient(
    url=QDRANT_API_URL,
    api_key=QDRANT_API_KEY,
    prefer_grpc=True
)

# Check if the collection exists
if not cloud_client.collection_exists("johnwick_parent"):
    cloud_client.create_collection(
        collection_name="johnwick_parent",
        vectors_config=models.VectorParams(
            size=1536,
            distance=models.Distance.COSINE
        ),
    )

# Construct the VectorStore using cloud client
parent_vectorstore = QdrantVectorStore(
    embedding=embeddings,
    client=cloud_client,
    collection_name="johnwick_parent",
)

store = InMemoryStore()

child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

parent_document_retriever.add_documents(parent_docs, ids=None)

In [11]:
# 3. Semantic chunking using qdrant cloud
semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

semantic_documents = semantic_chunker.split_documents(documents)

semantic_vectorstore = QdrantVectorStore.from_documents(
    semantic_documents,
    embeddings,
    url=QDRANT_API_URL,
    api_key=QDRANT_API_KEY,
    prefer_grpc=True,
    collection_name="johnwick_semantic"
)

## Retrievers

### Naive Retriever

In [12]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retriever = baseline_vectorstore.as_retriever(search_kwargs={"k" : 10})

naive_retrieval_chain = (
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | llm, "context": itemgetter("context")}
)

In [13]:
naive_retrieval_chain_response = naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

### BM25 Retriever

In [14]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | llm, "context": itemgetter("context")}
)

In [15]:
bm25_retrieval_chain_response = bm25_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

### Contextual Compression Retriever

In [16]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=naive_retriever
)

In [17]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | llm, "context": itemgetter("context")}
)

In [18]:
contextual_compression_retrieval_chain_response = contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

### Multi-Query Retriever



In [19]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever,
    llm=llm
)

In [20]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | llm, "context": itemgetter("context")}
)

In [21]:
multi_query_retrieval_chain_response = multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

### Parent Document Retriever

In [22]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | llm, "context": itemgetter("context")}
)

In [23]:
parent_document_retrieval_chain_response = parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

### Ensemble Retriever

In [24]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]

equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list,
    weights=equal_weighting
)

ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | llm, "context": itemgetter("context")}
)

In [25]:
ensemble_retrieval_chain_response = ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

### Semantic Retriever - using semantically chunked vector store

In [26]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | llm, "context": itemgetter("context")}
)

In [27]:
semantic_retrieval_chain_response = semantic_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

## Utilities

In [28]:
# display existing collections

existing = [c.name for c in cloud_client.get_collections().collections]

print(type(existing))
print(existing)

<class 'list'>
['johnwick_baseline', 'johnwick_semantic', 'johnwick_parent', 'airbnb_pdf_rec_1000_200_images', 'mcp-anthropic-desktop', 'mcp-nsclc', 'ambrose_lake_covenant']


In [29]:
# display vector store collection metadata

stores = {
    "baseline": baseline_vectorstore,
    "parent":  parent_vectorstore,
    "semantic": semantic_vectorstore,
}

for name, vs in stores.items():
    client = vs.client
    col    = vs.collection_name
    print(f"=== {name} ===")
    # 1) Existence check
    print("Exists?      ", client.collection_exists(col))
    # 2) Point count
    print("Point count: ", client.count(collection_name=col))
    # 3) Full collection info
    desc   = client.get_collection(collection_name=col)
    params = desc.config.params

    # — Vector dims & metric
    vec_field = params.vectors
    if isinstance(vec_field, dict):
        # multi-vector mode: pick the first VectorParams
        vp = next(iter(vec_field.values()))
    else:
        # single-vector mode: vectors is itself a VectorParams
        vp = vec_field
    print("Dim / metric:", vp.size, "/", vp.distance)

    # — Shard count & replication factor live on params
    print("Shards / repl:", params.shard_number, "/", params.replication_factor)

    print()


=== baseline ===
Exists?       True
Point count:  count=600
Dim / metric: 1536 / Cosine
Shards / repl: 1 / 1

=== parent ===
Exists?       True
Point count:  count=28902
Dim / metric: 1536 / Cosine
Shards / repl: 1 / 1

=== semantic ===
Exists?       True
Point count:  count=1074
Dim / metric: 1536 / Cosine
Shards / repl: 1 / 1



In [30]:
# Assume `store` already has docs via ParentDocumentRetriever
#  (i.e. you already did retriever.add_documents(...) or similar)

# 1) List all stored keys (document IDs)
all_keys = list(store.yield_keys())
print(f"Total documents in store: {len(all_keys)}")
# print("Document IDs:", all_keys)

# 2) Fetch all Document objects
docs = store.mget(all_keys)

# 3) Examine metadata schema
#    Collect all metadata field names across docs
all_fields = set()
for doc in docs:
    all_fields.update(doc.metadata.keys())

print(f"Metadata fields present: {sorted(all_fields)}")

# 4) Show per-field value types and a sample value
field_types = {field: set() for field in all_fields}
for doc in docs:
    for field, val in doc.metadata.items():
        field_types[field].add(type(val).__name__)

print("Metadata field types:")
for field, types in field_types.items():
    sample = next((d.metadata[field] for d in docs if field in d.metadata), None)
    print(f" • {field}: types={sorted(types)}, sample={sample!r}")

# 5) (Optional) Print out first N docs’ text lengths to gauge “dimensions”
for i, doc in enumerate(docs[:5], 1):
    text_len = len(doc.page_content)
    print(f"Doc {i} (ID={all_keys[i-1]}): {text_len} characters")



Total documents in store: 100
Metadata fields present: ['Author', 'Movie_Title', 'Rating', 'Review_Date', 'Review_Title', 'Review_Url', 'last_accessed_at', 'row', 'source']
Metadata field types:
 • Rating: types=['int'], sample=8
 • Movie_Title: types=['str'], sample='John Wick 1'
 • Review_Title: types=['str'], sample=' Kinetic, concise, and stylish; John Wick kicks ass.\n'
 • Review_Date: types=['str'], sample='6 May 2015'
 • Review_Url: types=['str'], sample='/review/rw3233896/?ref_=tt_urv'
 • source: types=['str'], sample='john_wick_1.csv'
 • Author: types=['str'], sample='lnvicta'
 • row: types=['int'], sample=0
 • last_accessed_at: types=['datetime'], sample=datetime.datetime(2025, 5, 20, 19, 23, 16, 383052)
Doc 1 (ID=0b0d1424-4259-4a60-992e-a413df1b09a7): 599 characters
Doc 2 (ID=6e32b3e3-e149-4303-80f7-857b1d67f264): 369 characters
Doc 3 (ID=f38a992d-0811-418b-a990-88f28909419b): 256 characters
Doc 4 (ID=7f9a1c9d-2816-4203-bf68-b32a17766b03): 426 characters
Doc 5 (ID=1de51d1f-5

### Delete qdrant collections

In [31]:
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, VectorParams

# initialize client (cloud or on-prem)
cloud_client = QdrantClient(
    url=QDRANT_API_URL,
    api_key=QDRANT_API_KEY,
    prefer_grpc=True,
)

# create conditional deletion flag
delete_collection = False

if delete_collection:
    # list of collections to drop
    collections_to_reset = [
        "johnwick_baseline",
        "johnwick_parent",
        "johnwick_semantic",
    ]

    for col_name in collections_to_reset:
        # guard against missing collections
        if cloud_client.collection_exists(col_name):
            cloud_client.delete_collection(
                collection_name=col_name,
                timeout=60,  # seconds
            )
            print(f"Deleted collection: {col_name}")
        else:
            print(f"Collection not found (skipped): {col_name}")


## Response Objects

In [32]:
from IPython.display import Markdown, display

# Map of titles to response objects
responses = {
    "Naive Retrieval Chain Response":              naive_retrieval_chain_response,
    "BM25 Retrieval Chain Response":               bm25_retrieval_chain_response,
    "Contextual Compression Chain Response":       contextual_compression_retrieval_chain_response,
    "Multi-Query Retrieval Chain Response":        multi_query_retrieval_chain_response,
    "Parent Document Retrieval Chain Response":    parent_document_retrieval_chain_response,
    "Ensemble Retrieval Chain Response":           ensemble_retrieval_chain_response,
    "Semantic Retrieval Chain Response":           semantic_retrieval_chain_response,
}

for header, resp in responses.items():
    display(Markdown(f"## {header}\n"))
    print("\n")
    print(resp)
    print("\n")


## Naive Retrieval Chain Response




Yes, people generally liked John Wick. The reviews highlight Keanu Reeves' slick performance, brilliant and meticulously choreographed action sequences, and the film's intense, fun, and violent style. Reviewers praised it as one of the best action films of the year and even the past decade, recommending it highly to action fans and those looking for something fresh. The movie was described as well-paced, stylish, and entertaining, with a strong supporting cast adding interest. Overall, the reception was very positive.




## BM25 Retrieval Chain Response




People's opinions on John Wick vary depending on the installment:

- The first John Wick movie is generally liked and praised. Reviews describe it as a special and stylish action film with smooth sequences, clear motives, and a relatable hero. It is recommended highly for action fans and seen as a must-watch. (Ratings like 8 and 10 support this.)

- However, later installments receive mixed to negative reviews. For example, John Wick 3 has been criticized as boring, dull, overly violent, and lacking plot.

- John Wick 4 is considered the weakest by at least one reviewer, described as nearly three hours of gunfights with meaningless dialogue and no real addition to the story.

In summary, the first John Wick film was generally well-liked, but later movies in the series received more negative and mixed reactions from some viewers.




## Contextual Compression Chain Response


...



I don't know.




## Ensemble Retrieval Chain Response




Yes, people generally liked the first John Wick movie. Reviews praise Keanu Reeves' performance, the slick and well-choreographed action sequences, and the stylish, fun nature of the film. It is described as one of the best action films in recent years, highly recommended especially for action fans. However, the reception of later films in the series is more mixed, with some reviewers expressing disappointment with the third and fourth installments. But overall, the original John Wick was well-received and enjoyed by many.




## Semantic Retrieval Chain Response




Yes, based on the reviews provided, people generally liked John Wick. One review describes the first John Wick movie as "something special," praising its elaborate action sequences, world-building, and smooth action scenes. It highlights Keanu Reeves' natural performance and the engaging criminal underworld setting. Additionally, multiple mentions state that "John Wick was cool," indicating a positive reception. Overall, the feedback suggests that audiences appreciated the film.




## Retrieval Chain visualizations

In [33]:
!uv pip install -qU grandalf

In [34]:
from IPython.display import Markdown, display

# Map of titles to chains
chains = {
    "Naive Retrieval":              naive_retrieval_chain,
    "BM25 Retrieval":               bm25_retrieval_chain,
    "Contextual Compression":       contextual_compression_retrieval_chain,
    "Multi-Query Retrieval":        multi_query_retrieval_chain,
    "Parent Document Retrieval":    parent_document_retrieval_chain,
    "Ensemble Retrieval":           ensemble_retrieval_chain,
    "Semantic Retrieval":           semantic_retrieval_chain,
}

for title, chain in chains.items():
    display(Markdown(f"## {title}\n"))
    print(chain.get_graph().draw_ascii())
    print("\n")


## Naive Retrieval


          +---------------------------------+      
          | Parallel<context,question>Input |      
          +---------------------------------+      
                    **            **               
                  **                **             
                **                    **           
         +--------+                     **         
         | Lambda |                      *         
         +--------+                      *         
              *                          *         
              *                          *         
              *                          *         
  +----------------------+          +--------+     
  | VectorStoreRetriever |          | Lambda |     
  +----------------------+          +--------+     
                    **            **               
                      **        **                 
                        **    **                   
          +----------------------------------+     
          | 

## BM25 Retrieval


          +---------------------------------+    
          | Parallel<context,question>Input |    
          +---------------------------------+    
                    **           **              
                  **               **            
                **                   **          
         +--------+                    **        
         | Lambda |                     *        
         +--------+                     *        
              *                         *        
              *                         *        
              *                         *        
      +---------------+            +--------+    
      | BM25Retriever |            | Lambda |    
      +---------------+            +--------+    
                    **           **              
                      **       **                
                        **   **                  
         +----------------------------------+    
         | Parallel<context,question>Output |    


## Contextual Compression


...

          +---------------------------------+      
          | Parallel<context,question>Input |      
          +---------------------------------+      
                    ***           ***              
                  **                 **            
                **                     **          
         +--------+                      **        
         | Lambda |                       *        
         +--------+                       *        
              *                           *        
              *                           *        
              *                           *        
+-------------------------+          +--------+    
| ParentDocumentRetriever |          | Lambda |    
+-------------------------+          +--------+    
                    ***           ***              
                       **       **                 
                         **   **                   
          +----------------------------------+     
          | 

## Ensemble Retrieval


          +---------------------------------+    
          | Parallel<context,question>Input |    
          +---------------------------------+    
                    **           **              
                  **               **            
                **                   **          
         +--------+                    **        
         | Lambda |                     *        
         +--------+                     *        
              *                         *        
              *                         *        
              *                         *        
    +-------------------+          +--------+    
    | EnsembleRetriever |          | Lambda |    
    +-------------------+          +--------+    
                    **           **              
                      **       **                
                        **   **                  
         +----------------------------------+    
         | Parallel<context,question>Output |    


## Semantic Retrieval


          +---------------------------------+      
          | Parallel<context,question>Input |      
          +---------------------------------+      
                    **            **               
                  **                **             
                **                    **           
         +--------+                     **         
         | Lambda |                      *         
         +--------+                      *         
              *                          *         
              *                          *         
              *                          *         
  +----------------------+          +--------+     
  | VectorStoreRetriever |          | Lambda |     
  +----------------------+          +--------+     
                    **            **               
                      **        **                 
                        **    **                   
          +----------------------------------+     
          | 